# Import Packages

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [106]:
## need to get Sciq data in the right format. waiting on Balaji

In [107]:
import os
import csv

In [113]:
os.getcwd()

'/content'

train_data = pd.read_csv("sample_data/train_mod.csv",
                         quotechar = '"'
                        #  quoting=csv.QUOTE_ALL,
# on_bad_lines='skip'
)

In [115]:
train_data = pd.read_json("sample_data/train.json")

ValueError: ignored

In [ ]:
train_data

In [ ]:
test_data = pd.read_json("sample_data/test.json")

In [ ]:
# df_1 = df[['support', 'correct_answer']]
# df_1['target'] = 'Similar'

# df_2 = df[['support', 'distractor1']]
# df_2['target'] = 'Dissimilar'

# pd.concat([df_1, df_2])

In [ ]:
df_ca = train_data.loc[:,['support','correct_answer']]
df_ca['target'] = 'Similar'
df_da1 = train_data.loc[:,['support','distractor1']].rename(columns = {'distractor1':'correct_answer'})
df_da1['target'] = 'Dissimilar'
pd.concat([df_ca,df_da1])

In [ ]:
df_ca = train_data.loc[:,['support','correct_answer']].rename(columns={'correct_answer':'response'})
df_ca['target'] = 'Similar'
df_da1 = train_data.loc[:,['support','distractor1']].rename(columns = {'distractor1':'response'})
df_da1['target'] = 'Dissimilar'
df_da2 =train_data.loc[:,['support','distractor2']].rename(columns = {'distractor2':'response'})
df_da2['target'] = 'Dissimilar'
df_da3 =train_data.loc[:,['support','distractor3']].rename(columns = {'distractor3':'response'})
df_da3['target'] = 'Dissimilar'

In [ ]:
df_result = pd.concat([df_ca,df_da1,df_da2,df_da3])

In [ ]:
df_result.shape

In [ ]:
df_result.iloc[0]['support']

In [ ]:
df_result.loc[df_result['support'] == "Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine."]

In [ ]:
df_result.head()

In [ ]:
train_data.head(6)

# Preprocessing

In [ ]:
# print(stop_words)

In [ ]:
include_words = set([''])

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

from tqdm import tqdm

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(data))
    return cleantext

def stripunc(data):
#     return re.sub('[^A-Za-z]+', ' ', str(data), flags=re.MULTILINE|re.DOTALL)
    return re.sub('[.,;!?\\-\'()]', ' ', str(data), flags=re.MULTILINE|re.DOTALL)


def compute(sent):

    sent = decontracted(sent)
#     sent = striphtml(sent)
    sent = stripunc(sent)

    words=word_tokenize(str(sent.lower()))
    include_words = set()

    #Removing all single letter and and stopwords from question
#     sent1=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1))
#     sent2=' '.join(str(j) for j in words if j not in stop_words and (len(j)!=1))
    sent1 =' '.join(str(stemmer.stem(j))  for j in words )
    sent2 =' '.join(str(j)  for j in words )
    return sent1, sent2

In [ ]:
df_temp_support = pd.DataFrame(df_result.loc[:,'support'].apply(compute).to_list(),index = df_result.index)

df_result['support_stemmed'] = df_temp_support[0]
df_result['support_unstemmed'] = df_temp_support[1]

df_temp_response = pd.DataFrame(df_result.loc[:,'response'].apply(compute).to_list(),index = df_result.index)
df_result['response_stemmed'] = df_temp_response[0]
df_result['response_unstemmed'] = df_temp_response[1]

df_result

In [ ]:
combined_unstemmed_text = list(df_result['support_unstemmed'].str.cat(' '+df_result['response_unstemmed']))

In [ ]:
len(combined_unstemmed_text)

In [ ]:
combined_org_text = list(df_result['support'].str.cat(' '+df_result['response']))

In [ ]:
len(combined_org_text)

In [ ]:
# combined_org_text = list(df_result['support'].str.cat(';;'+df_result['response']).iloc[0])

In [ ]:
df_result.shape

In [ ]:
df_result['combined_unstemmed_text'] = combined_unstemmed_text
df_result['combined_org_text'] = combined_org_text

In [ ]:
# test_data['clean_stemmed_q1_t'] = clean_stemmed_q1_t
# test_data['clean_stemmed_q2_t'] = clean_stemmed_q2_t
# test_data['clean_q1_t'] = clean_q1_t
# test_data['clean_q2_t'] = clean_q2_t
# test_data['combined_stemmed_text_t'] = combined_stemmed_text_t

test_data = pd.read_json("sample_data/test.json")

test_df_ca = test_data.loc[:,['support','correct_answer']].rename(columns={'correct_answer':'response'})
test_df_ca['target'] = 'Similar'
test_df_da1 = test_data.loc[:,['support','distractor1']].rename(columns = {'distractor1':'response'})
test_df_da1['target'] = 'Dissimilar'
test_df_da2 =test_data.loc[:,['support','distractor2']].rename(columns = {'distractor2':'response'})
test_df_da2['target'] = 'Dissimilar'
test_df_da3 =test_data.loc[:,['support','distractor3']].rename(columns = {'distractor3':'response'})
test_df_da3['target'] = 'Dissimilar'

test_data = pd.concat([test_df_ca,test_df_da1,test_df_da2,test_df_da3])

test_data.head(2)

In [ ]:
df_test_support = pd.DataFrame(test_data.loc[:,'support'].apply(compute).to_list(),index = test_data.index)

test_data['support_stemmed'] = df_test_support[0]
test_data['support_unstemmed'] = df_test_support[1]

df_test_response = pd.DataFrame(test_data.loc[:,'response'].apply(compute).to_list(),index = test_data.index)
test_data['response_stemmed'] = df_test_response[0]
test_data['response_unstemmed'] = df_test_response[1]

In [ ]:
test_data.head()

In [ ]:
train_data = df_result.copy()
train_data.tail()

In [ ]:
# test_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# X_temp, X_test, y_temp, y_test = train_test_split(data[['clean_q1', 'clean_q2']], data['is_duplicate'], test_size=0.2, random_state=42)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

In [ ]:
# X_train = train_data[['support', 'response']]
# y_train = train_data['target']

# X_test = test_data[['clean_q1_t', 'clean_q2_t']]
# y_test = test_data['is_duplicate']

X_train = df_result[['support_unstemmed', 'response_unstemmed']]
y_train = df_result['target']

X_test = test_data[['support_unstemmed', 'response_unstemmed']]
y_test = test_data['target']

In [ ]:
print(X_train.shape)
print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train['text'] = X_train[['support_unstemmed','response_unstemmed']].apply(lambda x:str(x[0])+" "+str(x[1]), axis=1)

In [ ]:
X_train.head()

In [ ]:
import tensorflow as tf

In [ ]:
import keras
import keras.backend as K

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Embedding, LSTM
from keras.models import Model

In [ ]:
! pip install joblib

In [ ]:
import joblib

Going a bit off-track here from the medium article.

Instead, following a more up-to-date embedding approach as mentioned [here](https://keras.io/examples/nlp/pretrained_word_embeddings/).

***START OF NEW EMBEDDING APRROACH***

In [ ]:
# TextVectorization documentation here: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=10)
text_ds = tf.data.Dataset.from_tensor_slices((X_train['support_unstemmed'] + ' ' +  X_train['response_unstemmed']).values).batch(256)
vectorizer.adapt(text_ds)


In [ ]:
vectorizer.get_vocabulary()[:10]

In [ ]:
output = vectorizer([["share market history are airpl"]])
output.numpy()

# value 1 in the output represents out-of-voccabulary words
# the other integer values just represent the index position of the word in the vocabulary
# for eg capricorn is represented as 3 because it is at 4th place in the vocab list

In [ ]:
type(output)

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))


In [ ]:
word_index

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = 'glove.6B.100d.txt'

cnt = 0
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        if cnt < 5:
          print(line, '\n')
          cnt += 1
        word, coefs = line.split(maxsplit=1)
        #maxsplit = 1 means split on the first white space, which separates actual word on the left from the vector on the right
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


In [ ]:
type(embeddings_index['hello'])

Now, let's prepare a corresponding embedding matrix that we can use in a Keras Embedding layer. It's a simple NumPy matrix where entry at index i is the pre-trained vector for the word of index i in our vectorizer's vocabulary.

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0
missed_words=[]
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        missed_words.append(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
missed_words[:5]

In [ ]:
# Why are we doing this???


# #Trying feeding text directly into the embedding matrix without token vectorizer
# num_tokens = len(voc) + 2
# embedding_dim = 100
# hits = 0
# misses = 0

# embedding_matrix = np.zeros((num_tokens, embedding_dim))
# for word in X_train['text']:
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # Words not found in embedding index will be all-zeros.
#         # This includes the representation for "padding" and "OOV"
#         embedding_matrix[i] = embedding_vector
#         hits += 1
#     else:
#         misses += 1
# print("Converted %d words (%d misses)" % (hits, misses))

Next, we load the pre-trained word embeddings matrix into an Embedding layer.

Note that we set trainable=False so as to keep the embeddings fixed (we don't want to update them during training).

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)


***END OF NEW EMBEDDING APRROACH***

Now we start building Siamese using embeddings generated from new approach

In [ ]:
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Concatenate
from keras.layers import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D

In [ ]:
from keras import backend as K

def cosine_distance(vests):
  x, y = vests
  x = K.l2_normalize(x, axis=-1)
  y = K.l2_normalize(y, axis=-1)
  return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
  shape1, shape2 = shapes
  return (shape1[0],1)

In [ ]:
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
  return roc_auc_score(y_true, y_pred)


Now we need

1. String data of q1
2. Convert it into index-vector
3. Convert index-vector into embedding space

In [ ]:
X_train.shape

In [ ]:
arr = np.array([[s] for s in X_train['support_unstemmed']])
arr.shape

In [ ]:
index_vec_1 = vectorizer([s for s in X_train['support_unstemmed']])
index_vec_2 = vectorizer([s for s in X_train['response_unstemmed']])


# y_train is already defined


In [ ]:
y_train

In [ ]:
y_train.loc[y_train == 'Similar'] = 1
y_train.loc[y_train == 'Dissimilar'] = 0
y_train[:10]

In [ ]:
y_train.value_counts()

In [ ]:
y_test.loc[y_test == 'Similar'] = 1
y_test.loc[y_test == 'Dissimilar'] = 0
y_test.value_counts()

In [ ]:
index_vec_1[-1]

In [ ]:
X_train.tail(1)

In [ ]:
index_vec_1.shape

In [ ]:
index_vec_2.shape

In [ ]:
index_vec_2[-1]

In [ ]:
# for shape, we need to give the length of the index-vector for each question
# in this case, it should come out to be 10 and the shape will be (10,)
# we also could have left the sahpe as (None,) and the model would have inferred it from the acutal data
input_1 = Input(shape=(index_vec_1.shape[1],), dtype = 'int64')
input_2 = Input(shape=(index_vec_2.shape[1],), dtype = 'int64')

lstm_1 = embedding_layer(input_1)
lstm_2 = embedding_layer(input_2)


common_lstm = LSTM(64,return_sequences=True, activation="relu")
vector_1 = common_lstm(lstm_1)
vector_1 = Flatten()(vector_1)

vector_2 = common_lstm(lstm_2)
vector_2 = Flatten()(vector_2)

x3 = Subtract()([vector_1, vector_2])
x3 = Multiply()([x3, x3])

x1_ = Multiply()([vector_1, vector_1])
x2_ = Multiply()([vector_2, vector_2])
x4 = Subtract()([x1_, x2_])

    #https://stackoverflow.com/a/51003359/10650182
x5 = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([vector_1, vector_2])

conc = Concatenate(axis=-1)([x5,x4, x3])

x = Dense(100, activation="relu", name='conc_layer')(conc)
x = Dropout(0.01)(x)
out = Dense(1, activation="sigmoid", name = 'out')(x)

model = Model([input_1, input_2], out)

model.compile(loss="binary_crossentropy", metrics=['acc', keras.metrics.AUC(name='auc')], optimizer=Adam(0.00001))

In [ ]:
model.summary()

In [ ]:
y_train[:5]

In [ ]:
#Get and process validation data
# validn_data = pd.read_json("sample_data/valid.json")

# test_df_ca = validn.loc[:,['support','correct_answer']].rename(columns={'correct_answer':'response'})
# test_df_ca['target'] = 'Similar'
# test_df_da1 = test_data.loc[:,['support','distractor1']].rename(columns = {'distractor1':'response'})
# test_df_da1['target'] = 'Dissimilar'
# test_df_da2 =test_data.loc[:,['support','distractor2']].rename(columns = {'distractor2':'response'})
# test_df_da2['target'] = 'Dissimilar'
# test_df_da3 =test_data.loc[:,['support','distractor3']].rename(columns = {'distractor3':'response'})
# test_df_da3['target'] = 'Dissimilar'

# test_data = pd.concat([test_df_ca,test_df_da1,test_df_da2,test_df_da3])

valdn_vec_1 = vectorizer([s for s in X_test['support_unstemmed']])
valdn_vec_2 = vectorizer([s for s in X_test['response_unstemmed']])


In [ ]:
valdn_vec_1[:4]

In [ ]:
y_test[:2]

In [ ]:
y_test.values.reshape(-1,1).shape

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [ ]:
type(index_vec_2)

In [ ]:
tf.convert_to_tensor(y_train.values.reshape(-1,1).astype('float32'))

In [ ]:

model.fit([index_vec_1,index_vec_2],
          tf.convert_to_tensor(y_train.values.reshape(-1,1).astype('float32')),
          epochs = 5,
          batch_size=64,validation_data=([index_vec_1, index_vec_2],
                                         tf.convert_to_tensor(y_train.values.reshape(-1,1).astype('float32')))
          )